Problem-**1**

In [1]:
# Step 1: Install required packages
!pip install -q langchain-google-genai langchain-core tavily-python requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 713.3/713.3 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.9/234.9 kB 14.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.47.0 which is incompatible.


In [2]:
# Step 2: Import libraries

import requests
from tavily import TavilyClient
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from google.colab import userdata

GEMINI_API_KEY = userdata.get("Assignments")
OPENWEATHER_API_KEY = userdata.get("openweatherapi")
TAVILY_API_KEY = userdata.get("tavily")

In [3]:
def get_weather(city):
    """Fetch real-time weather data from OpenWeatherMap"""
    try:
        url = (
            f"https://api.openweathermap.org/data/2.5/weather"
            f"?q={city}&appid={OPENWEATHER_API_KEY}&units=metric"
        )
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        return {
            "temperature": data["main"]["temp"],
            "feels_like": data["main"]["feels_like"],
            "description": data["weather"][0]["description"],
            "humidity": data["main"]["humidity"],
            "wind_speed": data["wind"]["speed"]
        }
    except Exception as e:
        return {"error": str(e)}

In [4]:
def search_tavily(query, max_results=5):
    """Search travel-related information using Tavily"""
    try:
        client = TavilyClient(api_key=TAVILY_API_KEY)
        response = client.search(query=query, max_results=max_results)
        return response.get("results", [])
    except Exception as e:
        return {"error": str(e)}

In [5]:
def generate_itinerary(city, duration, interests, time_preference, weather_data, search_results):
    """Generate personalized travel itinerary using Gemini"""
    try:
        llm = ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-lite",
            temperature=0.7,
            google_api_key=GEMINI_API_KEY
        )

        weather_info = f"""
Weather in {city}:
- Temperature: {weather_data.get('temperature', 'N/A')}°C
- Feels like: {weather_data.get('feels_like', 'N/A')}°C
- Conditions: {weather_data.get('description', 'N/A')}
- Humidity: {weather_data.get('humidity', 'N/A')}%
- Wind Speed: {weather_data.get('wind_speed', 'N/A')} m/s
"""

        search_context = "\n\nRecommendations from search:\n"
        for i, item in enumerate(search_results[:10], 1):
            search_context += (
                f"{i}. {item.get('title', 'N/A')}\n"
                f"   {item.get('content', 'N/A')[:200]}...\n"
                f"   URL: {item.get('url', 'N/A')}\n\n"
            )

        system_prompt = SystemMessage(
            content="""You are an expert travel planner.
Create detailed, personalized, day-wise itineraries based on
user preferences, weather, and local recommendations."""
        )

        user_prompt = HumanMessage(
            content=f"""
Create a personalized {duration}-day travel itinerary for {city}.

User Preferences:
- Interests: {', '.join(interests)}
- Preferred time of day: {time_preference}

{weather_info}
{search_context}

Include:
1. Weather summary + packing tips
2. Top 5 attractions
3. 5 hotels (budget to luxury)
4. 5 restaurants
5. Detailed day-wise itinerary with timings and travel tips
"""
        )

        response = llm.invoke([system_prompt, user_prompt])
        return response.content

    except Exception as e:
        return f"Error generating itinerary: {e}"

In [6]:
def travel_assistant(city, duration, interests, time_preference):

    print("\n" + "=" * 60)
    print("🌍 AI TRAVEL ASSISTANT")
    print("=" * 60 + "\n")

    print(f"📍 Destination: {city}")
    print(f"📅 Duration: {duration} days")
    print(f"🎯 Interests: {', '.join(interests)}")
    print(f"🕐 Preferred time: {time_preference}\n")

    print("☁️ Fetching weather data...")
    weather_data = get_weather(city)
    if "error" in weather_data:
        print(f"⚠️ Weather error: {weather_data['error']}")
    else:
        print(f"✅ Weather: {weather_data['temperature']}°C, {weather_data['description']}\n")

    print("🔍 Fetching travel recommendations...")
    queries = [
        f"best attractions in {city}",
        f"top hotels in {city}",
        f"best restaurants in {city}",
        f"{city} travel guide {', '.join(interests)}"
    ]

    all_results = []
    for q in queries:
        results = search_tavily(q, max_results=3)
        if isinstance(results, list):
            all_results.extend(results)

    print(f"✅ Found {len(all_results)} recommendations\n")

    print("🤖 Generating itinerary using Gemini...\n")
    itinerary = generate_itinerary(
        city, duration, interests, time_preference, weather_data, all_results
    )

    print("=" * 60)
    print("📋 YOUR PERSONALIZED TRAVEL ITINERARY")
    print("=" * 60 + "\n")
    print(itinerary)
    print("\n✨ Happy travels! ✨\n")

    return itinerary


In [7]:
print("\n" + "=" * 60)
print("🌟 WELCOME TO AI TRAVEL ASSISTANT")
print("=" * 60 + "\n")

city = input("Enter destination city: ").strip()
duration = int(input("Enter trip duration (in days): ").strip())

print("\nEnter interests (comma-separated):")
interests = [i.strip() for i in input("Your interests: ").split(",")]

print("\nPreferred time of day:")
time_preference = input("Your preference: ").strip()

# Run assistant
travel_assistant(city, duration, interests, time_preference)



🌟 WELCOME TO AI TRAVEL ASSISTANT

Enter destination city: mumbai
Enter trip duration (in days): 4

Enter interests (comma-separated):
Your interests: fort,shoppping

Preferred time of day:
Your preference: day, night

🌍 AI TRAVEL ASSISTANT

📍 Destination: mumbai
📅 Duration: 4 days
🎯 Interests: fort, shoppping
🕐 Preferred time: day, night

☁️ Fetching weather data...
✅ Weather: 29.99°C, smoke

🔍 Fetching travel recommendations...
✅ Found 12 recommendations

🤖 Generating itinerary using Gemini...

📋 YOUR PERSONALIZED TRAVEL ITINERARY

Here is a personalized 4-day itinerary for your trip to Mumbai, focusing on your interests in Fort and shopping, while keeping the current weather in mind.

## Mumbai: A 4-Day Fort & Shopping Adventure

**Weather Summary & Packing Tips:**

Mumbai's weather is currently warm and humid with a smoky haze. Expect temperatures around 30°C (86°F) with a "feels like" temperature slightly lower due to humidity and a gentle breeze. The smoke haze suggests it might 

'Here is a personalized 4-day itinerary for your trip to Mumbai, focusing on your interests in Fort and shopping, while keeping the current weather in mind.\n\n## Mumbai: A 4-Day Fort & Shopping Adventure\n\n**Weather Summary & Packing Tips:**\n\nMumbai\'s weather is currently warm and humid with a smoky haze. Expect temperatures around 30°C (86°F) with a "feels like" temperature slightly lower due to humidity and a gentle breeze. The smoke haze suggests it might be best to limit prolonged outdoor activities during peak sun hours and consider wearing a mask if you have respiratory sensitivities.\n\n**Packing Essentials:**\n\n*   **Lightweight, breathable clothing:** Cotton, linen, and other natural fabrics will be most comfortable. Think loose-fitting tops, t-shirts, and comfortable trousers or skirts.\n*   **Sun protection:** A wide-brimmed hat, sunglasses, and high SPF sunscreen are essential.\n*   **Comfortable walking shoes:** You\'ll be doing a lot of exploring!\n*   **A light sca